In [1]:
import altair as alt
import eland as ed
import json
import urllib3

In [2]:
http = urllib3.PoolManager()
url = 'http://localhost:9200/farequote_eland_2/_search?size=2'
r = http.request('GET', url, headers=headers)
r.status

200

In [3]:
fq = json.loads(r.data.decode('utf-8'))['hits']['hits']

In [4]:
fq

[{'_index': 'farequote_eland_2',
  '_id': '500',
  '_score': 1.0,
  '_source': {'time': '2016-02-07T00:46:43+0000',
   'airline': 'JAL',
   'responsetime': 496.3348,
   'sourcetype': 'farequote'}},
 {'_index': 'farequote_eland_2',
  '_id': '501',
  '_score': 1.0,
  '_source': {'time': '2016-02-07T00:46:49+0000',
   'airline': 'VRD',
   'responsetime': 300.5123,
   'sourcetype': 'farequote'}}]

In [5]:
#ed.read_csv(
#    "farequote.csv",
#    es_client='http://localhost:9200',
#    es_dest_index='farequote_eland_2',
#    es_refresh=True
#)

In [6]:
df = ed.DataFrame('http://localhost:9200', 'farequote_eland_2')
print(df.info_es())

index_pattern: farequote_eland_2
Index:
 index_field: _id
 is_source_field: False
Mappings:
 capabilities:
             es_field_name  is_source es_dtype es_date_format pd_dtype  is_searchable  is_aggregatable  is_scripted aggregatable_es_field_name
airline            airline       True  keyword           None   object           True             True        False                    airline
responsetime  responsetime       True   double           None  float64           True             True        False               responsetime
sourcetype      sourcetype       True  keyword           None   object           True             True        False                 sourcetype
time                  time       True  keyword           None   object           True             True        False                       time
Operations:
 tasks: []
 size: None
 sort_params: None
 _source: ['airline', 'responsetime', 'sourcetype', 'time']
 body: {}
 post_processing: []



In [7]:
df.head()

,airline,responsetime,sourcetype,time
500,JAL,496.3348,farequote,2016-02-07T00:46:43+0000
501,VRD,300.5123,farequote,2016-02-07T00:46:49+0000
502,ACA,14.2544,farequote,2016-02-07T00:46:57+0000
503,FFT,204.6499,farequote,2016-02-07T00:46:59+0000
504,JAL,490.7523,farequote,2016-02-07T00:47:03+0000


In [8]:
df = df[['airline', 'responsetime', 'time']]

In [9]:
df[['airline', 'responsetime']]

,airline,responsetime
500,JAL,4.963348e+02
501,VRD,3.005123e+02
502,ACA,1.425440e+01
503,FFT,2.046499e+02
504,JAL,4.907523e+02
...,...,...
86270,JBU,9.236772e+02
86271,ACA,2.153850e+01
86272,FFT,2.515730e+02
86273,ASA,7.829270e+01


In [10]:
df[['responsetime']].aggregate(['sum', 'min', 'std'])

,responsetime
sum,4.851854e+07
min,4.252500e+00
std,4.049250e+03


In [11]:
df.shape

(86275, 3)

In [12]:
df[['responsetime']].agg('sum')

,responsetime
sum,48518537.18


In [13]:
pd_df = ed.eland_to_pandas(df)

In [21]:
pd_df = pd_df.head(10000)

In [22]:
pd_df.shape

(10000, 3)

In [23]:
pd_df.head()

,airline,responsetime,time
500,JAL,496.3348,2016-02-07T00:46:43+0000
501,VRD,300.5123,2016-02-07T00:46:49+0000
502,ACA,14.2544,2016-02-07T00:46:57+0000
503,FFT,204.6499,2016-02-07T00:46:59+0000
504,JAL,490.7523,2016-02-07T00:47:03+0000


In [24]:
alt.data_transformers.disable_max_rows()
alt.Chart(pd_df).mark_point(filled=True, size=10, opacity=0.5).encode(
    x='time:T',
    y='responsetime:Q'
).properties(
    width=720,
    height=300
).interactive()

alt.Chart(...)

In [25]:
alt.Chart(pd_df).mark_boxplot(extent="min-max").encode(
    x='responsetime:Q',
    y='airline:O',
    tooltip=dict({
      "field": "airline",
      "type": "quantitative"
    })
).properties(
    width=650,
    height=300
)

alt.Chart(...)

In [26]:
data = alt.UrlData(
    url='http://127.0.0.1:9200/farequote_eland_2/_search?size=4999',
    format=dict({ "property": "hits.hits" })
)

alt.Chart(data).mark_point(filled=True, size=10, opacity=0.5).encode(
    x='_source.time:T',
    y='_source.responsetime:Q',
    tooltip=['_source.airline:N', '_source.responsetime:Q']
).properties(
    width=600,
    height=300
).interactive()

alt.Chart(...)

In [20]:
data

UrlData({
  format: {'property': 'hits.hits'},
  url: 'http://127.0.0.1:9200/farequote_eland_2/_search?size=4999'
})